In [1]:
import pandas as pd
import numpy as np

from sys import path
path.append('../py/')
import utils

In [2]:
ll ../data/

total 105651932
-rw-rw-r-- 1 Kazuki    24819734 Oct 30 06:57 oid_exgal.pkl
-rw-rw-r-- 1 Kazuki     3124774 Oct 30 06:57 oid_gal.pkl
-rw-rw-r-- 1 Kazuki       63462 Oct 30 06:57 target.pkl
-rw-rw-r-- 1 Kazuki   279432339 Oct 30 08:00 test_f001.pkl
-rw-rw-r-- 1 Kazuki   838296109 Oct 30 11:26 test_f002.pkl
-rw-rw-r-- 1 Kazuki 17604223124 Oct 30 12:51 test_f003.pkl
-rw-rw-r-- 1 Kazuki  5029776406 Oct 30 12:37 test_f004.pkl
-rw-rw-r-- 1 Kazuki 15089323152 Oct 31 18:43 test_f005.pkl
-rw-rw-r-- 1 Kazuki   838296109 Oct 31 17:42 test_f006.pkl
-rw-rw-r-- 1 Kazuki   838296109 Oct 31 12:51 test_f007.pkl
-rw-rw-r-- 1 Kazuki 36884968726 Oct 31 15:39 test_f008.pkl
-rw-rw-r-- 1 Kazuki   726523688 Oct 31 16:37 test_fbk1-002.pkl
-rw-rw-r-- 1 Kazuki  3143608240 Oct 31 16:54 test_fbk1-004.pkl
-rw-rw-r-- 1 Kazuki   265341414 Oct 30 07:05 test_log00.pkl
-rw-rw-r-- 1 Kazuki   264644022 Oct 30 07:06 test_log01.pkl
-rw-rw-r-- 1 Kazuki   260844848 Oct 30 07:06 test_log02.pkl
-rw-rw-r-- 1 Kazuki   261781200 Oc

In [50]:
y_true = pd.read_pickle('../data/y_true.pkl').values
y_pred = pd.read_pickle('../data/y_pred.pkl').values

In [54]:
y_pred = y_pred.astype(float)

In [13]:
y_pred

array([[0.0009505958376533336, 0.00025884967702269134,
        0.08371499526789983, ..., 0.00037782096681084553,
        0.9084327963237997, 0.00016212052432236845],
       [0.0005430252310503553, 0.002980356597897322,
        0.0037195013733742495, ..., 0.005165907624154909,
        0.0010706449495498983, 0.0016684597149904555],
       [0.001224858331589871, 0.00037114662207022187,
        0.0003382994055490172, ..., 0.1793990633515133,
        0.0013189531107712599, 0.0011604345213289751],
       ...,
       [0.006358725512743429, 0.0003764714901848215, 0.9014462576419001,
        ..., 0.0007094698153977152, 0.0012496005980834485,
        0.00015143204877152746],
       [0.00012920061031243085, 0.0004389285996862273,
        9.057957440755234e-05, ..., 0.00021741054223480078,
        0.00012116956155298362, 2.3922582752609507e-05],
       [0.9067834269484193, 0.005286130349579469, 0.001962580494436689,
        ..., 0.018632330044072627, 0.035331341750558416,
        0.003406466925683

In [4]:
oid_gal   = pd.read_pickle('../data/oid_gal.pkl').object_id
oid_exgal = pd.read_pickle('../data/oid_exgal.pkl').object_id

In [5]:
def multi_weighted_logloss(y_true, y_pred, myweight=None):
    """
    @author olivier https://www.kaggle.com/ogrellier
    multi logloss for PLAsTiCC challenge
    """
    # class_weights taken from Giba's topic : https://www.kaggle.com/titericz
    # https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
    # with Kyle Boone's post https://www.kaggle.com/kyleboone
    classes = [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]
    class_weight = {6: 1, 15: 2, 16: 1, 42: 1, 52: 1, 53: 1, 62: 1, 64: 2, 65: 1, 67: 1, 88: 1, 90: 1, 92: 1, 95: 1}
    if y_true.shape[1] > 14:
        classes.append(99)
        class_weight[99] = 2
    
    if myweight is None:
        myweight = np.ones(M)
    y_p = y_pred * myweight
    
    # normalize
    y_p /= y_p.sum(1)[:,None]
    
    # Normalize rows and limit y_preds to 1e-15, 1-1e-15
    y_p = np.clip(a=y_p, a_min=1e-15, a_max=1 - 1e-15)
    # Transform to log
    y_p_log = np.log(y_p)
    # Get the log for ones, .values is used to drop the index of DataFrames
    # Exclude class 99 for now, since there is no class99 in the training set
    # we gave a special process for that class
    y_log_ones = np.sum(y_true * y_p_log, axis=0)
    # Get the number of positives for each class
    nb_pos = y_true.sum(axis=0).astype(float)
    # Weight average and divide by the number of positives
    class_arr = np.array([class_weight[k] for k in sorted(class_weight.keys())])
    y_w = y_log_ones * class_arr / nb_pos

    loss = - np.sum(y_w) / np.sum(class_arr)
    return loss


def calc_gradient(f, X):
    """
    calc_gradient
    偏微分を行う関数
    関数fを変数xの各要素で偏微分した結果をベクトルにした勾配を返す
    
    @params
    f: 対象となる関数
    X: 関数fの引数のベクトル(numpy.array)
    
    @return
    gradient: 勾配(numpy.array)
    """
    
    h = 1e-4
    gradient = np.zeros_like(X)
    
    # 各変数についての偏微分を計算する
    for i in range(X.size):
        store_X = X[:]
        
        # f(x+h)
        X[i] += h
        f_x_plus_h = f(X)

        X = store_X[:]
        
        # f(x-h)
        X[i] -= h
        f_x_minus_h = f(X)
        
        # 偏微分
        gradient[i] = (f_x_plus_h - f_x_minus_h) / (2 * h)
        
    return gradient

def gradient_descent(f, X, learning_rate, max_iter):
    """
    gradient_descent
    最急降下法を行う関数
    
    @params
    f: 対象となる関数
    X: 関数fの引数のベクトル(numpy.array)
    learning_rate: 学習率
    max_iter: 繰り返し回数
    
    @return
    X: 関数の出力を最小にする(であろう)引数(numpy.array)
    """
    
    for i in range(max_iter):
        X -= (learning_rate * calc_gradient(f, X))
        if i%50==0:
            print("[{:3d}] X = {}, f(X) = {:.7f}".format(i, X, f(X)))
        
    return X

In [6]:
M = 14

X = np.ones(M)

In [11]:
y_true.shape, y_pred.shape

((7848, 14), (7848, 14))

In [56]:
f = lambda X: multi_weighted_logloss(y_true, y_pred, X)
gradient_descent(f, X, learning_rate=0.1, max_iter=1000)


[  0] X = [1.02031787 1.08416676 0.25167659 0.20050711 1.85427079 1.50502924
 0.52488637 3.05258952 0.19932094 1.463357   0.65343934 0.11475691
 0.59903887 1.25227981], f(X) = 0.6208683
[ 50] X = [1.01942445 1.08155556 0.25127425 0.18195751 1.85193459 1.50507269
 0.52068427 3.06033257 0.19906193 1.46112444 0.65181473 0.07840132
 0.59714247 1.25160852], f(X) = 0.6179282
[100] X = [1.01842369 1.07801999 0.25081318 0.18056766 1.84851848 1.50487844
 0.5159042  3.06769345 0.19873655 1.45800572 0.65006281 0.07812649
 0.59519545 1.25045271], f(X) = 0.6178792
[150] X = [1.0173954  1.07454725 0.25035218 0.18004903 1.84502183 1.50461535
 0.51231021 3.07488284 0.19844989 1.45482601 0.64832009 0.07793879
 0.59328683 1.24921598], f(X) = 0.6178357
[200] X = [1.01634758 1.07117926 0.24989578 0.17962146 1.84148876 1.50429469
 0.50960137 3.08191884 0.19819528 1.45163545 0.64659291 0.07777147
 0.59141982 1.24792296], f(X) = 0.6177956
[250] X = [1.01528509 1.06792873 0.2494462  0.17924325 1.83793895 1.50

array([0.99890379, 1.03253755, 0.24371138, 0.17544405, 1.78726454,
       1.49413711, 0.49543226, 3.1784595 , 0.1955218 , 1.40734184,
       0.6220276 , 0.07591669, 0.56700101, 1.2241892 ])

In [64]:
X

array([0.99890379, 1.03253755, 0.24371138, 0.17544405, 1.78726454,
       1.49413711, 0.49543226, 3.1784595 , 0.1955218 , 1.40734184,
       0.6220276 , 0.07591669, 0.56700101, 1.2241892 ])

In [65]:
X = np.append(X, 1)

In [71]:
sub = pd.read_csv('../output/1022-1_Giba-post2.csv.gz')

In [72]:
sub.head()

,object_id,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99
0,13,0.0,0.000476,0.0,0.618422,0.085550,0.0,0.021970,0.000197,0.0,0.001204,0.000679,0.265750,0.0,0.001734,0.17
1,14,0.0,0.002630,0.0,0.238392,0.020065,0.0,0.008808,0.002570,0.0,0.002217,0.023524,0.686737,0.0,0.008908,0.17
2,17,0.0,0.002050,0.0,0.069637,0.004770,0.0,0.004623,0.001582,0.0,0.004237,0.022567,0.874976,0.0,0.011598,0.17
3,23,0.0,0.000667,0.0,0.056746,0.011112,0.0,0.018011,0.000856,0.0,0.046969,0.003459,0.846949,0.0,0.011036,0.17
4,34,0.0,0.000670,0.0,0.045704,0.070154,0.0,0.006544,0.000161,0.0,0.000827,0.000197,0.872713,0.0,0.000470,0.17


In [73]:
val = sub.iloc[:, 1:].values

In [74]:
val *= X

In [75]:
val /= val.sum(1)[:, None]

In [76]:
sub.iloc[:, 1:] = val

In [77]:
sub.to_csv('../output/1022-1_Giba-post2-1103.csv.gz', index=False, compression='gzip')

In [78]:
utils.submit('../output/1022-1_Giba-post2-1103.csv.gz', 'only gal')